# Thực hiện thống kê mô tả trên tập dữ liệu về phân loại chất lượng rượu đỏ.
Phân tích này nhằm tính toán các thước đo thống kê mô tả (trung bình, trung vị, mode, phương sai, độ lệch chuẩn, min, max, phân vị, IQR) để hiểu rõ hơn về đặc điểm của dữ liệu rượu đỏ, đặc biệt là cột `quality` và các thuộc tính đầu vào.

## 1. Import thư viện và nạp dữ liệu vào notebook.


In [1]:
import numpy as np
import pandas as pd
from IPython import display
from scipy import stats

# Tải dữ liệu
df_dataset = pd.read_csv('winequality-red.csv')

# Hiển thị 5 dòng đầu tiên
display.display(df_dataset.head())

# Kiểm tra kiểu dữ liệu
print('Kiểu dữ liệu của các cột:')
print(df_dataset.dtypes)

# Kiểm tra kích thước dữ liệu
print(f'Kích thước dữ liệu: {df_dataset.shape}')


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Kiểu dữ liệu của các cột:
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object
Kích thước dữ liệu: (1599, 12)


**Nhận xét:**
- Tập dữ liệu gồm 1599 dòng và 12 cột, với 11 cột đầu vào (input features) và 1 cột `quality` biểu thị chất lượng rượu đỏ (từ 0 đến 10).
- Tất cả các cột đều là kiểu số (`float64` cho các thuộc tính đầu vào, `int64` cho `quality`), phù hợp cho phân tích thống kê.
- Các thuộc tính đầu vào bao gồm các đặc điểm hóa học như độ axit, hàm lượng đường, mật độ, v.v., ảnh hưởng đến chất lượng rượu.

## 2. Thống kê mô tả tổng quan bằng `describe()`
Sử dụng phương thức `describe()` để có cái nhìn tổng quan về các thước đo thống kê của tất cả cột số.

In [2]:
print('Thống kê mô tả tổng quan:')
display.display(df_dataset.describe())


Thống kê mô tả tổng quan:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


**Nhận xét về `describe()`**:
- Cột `quality`: Trung bình ~5.64, trung vị ~6, min=3, max=8, độ lệch chuẩn ~0.81, cho thấy chất lượng rượu tập trung quanh mức 5-6.
- Các cột như `fixed acidity`, `volatile acidity`, v.v., có phạm vi và độ phân tán khác nhau, ví dụ `total sulfur dioxide` có độ lệch chuẩn cao (~32.9), cho thấy sự biến động lớn.
- Một số cột như `chlorides` có giá trị min/max rất chênh lệch, gợi ý có thể có ngoại lai.

## 3. Thống kê chi tiết cho cột `quality`
Tính các thước đo thống kê cụ thể cho cột `quality`.

In [7]:
# Trung bình
df_mean = np.mean(df_dataset['quality'])
print(f'Trung bình của quality: {df_mean:.2f}')

# Trung vị
df_median = np.median(df_dataset['quality'])
print(f'Trung vị của quality: {df_median}')

# Mode
df_mode = stats.mode(df_dataset['quality'])  # Lấy giá trị mode
df_mode_count = stats.mode(df_dataset['quality']) # Lấy số lần xuất hiện
print(f'Mode của quality: {df_mode} (xuất hiện {df_mode_count} lần)')

# Phương sai
df_variance = np.var(df_dataset['quality'], ddof=1)  # ddof=1 để tính phương sai mẫu
print(f'Phương sai của quality: {df_variance:.2f}')

# Độ lệch chuẩn
df_std = np.std(df_dataset['quality'], ddof=1)  # ddof=1 để tính độ lệch chuẩn mẫu
print(f'Độ lệch chuẩn của quality: {df_std:.2f}')

# Min, Max
df_max = np.max(df_dataset['quality'])
df_min = np.min(df_dataset['quality'])
print(f'Giá trị lớn nhất của quality: {df_max}')
print(f'Giá trị nhỏ nhất của quality: {df_min}')

# Phân vị thứ 60
df_percentile = np.percentile(df_dataset['quality'], 60)
print(f'Phân vị thứ 60 của quality: {df_percentile}')

# Tứ phân vị thứ ba (Q3)
df_quartile = np.quantile(df_dataset['quality'], 0.75)
print(f'Tứ phân vị thứ ba (Q3) của quality: {df_quartile}')

# Khoảng tứ phân vị (IQR)
df_IQR = stats.iqr(df_dataset['quality'])
print(f'Khoảng tứ phân vị (IQR) của quality: {df_IQR}')


Trung bình của quality: 5.64
Trung vị của quality: 6.0
Mode của quality: ModeResult(mode=np.int64(5), count=np.int64(681)) (xuất hiện ModeResult(mode=np.int64(5), count=np.int64(681)) lần)
Phương sai của quality: 0.65
Độ lệch chuẩn của quality: 0.81
Giá trị lớn nhất của quality: 8
Giá trị nhỏ nhất của quality: 3
Phân vị thứ 60 của quality: 6.0
Tứ phân vị thứ ba (Q3) của quality: 6.0
Khoảng tứ phân vị (IQR) của quality: 1.0


**Nhận xét về cột `quality`**:
- **Trung bình (5.64)**: Chất lượng rượu trung bình ở mức khá, gần 6/10.
- **Trung vị (6.0)**: Gần với trung bình, cho thấy phân phối khá cân đối, không bị lệch mạnh.
- **Mode (5, xuất hiện 681 lần)**: Chất lượng 5 là phổ biến nhất, chiếm tỷ lệ lớn trong dữ liệu.
- **Phương sai (0.65) và độ lệch chuẩn (0.81)**: Độ phân tán của chất lượng không quá lớn, các giá trị tập trung quanh mức 5-6.
- **Min (3), Max (8)**: Chất lượng rượu dao động từ kém (3) đến rất tốt (8), nhưng không có rượu đạt điểm 9 hoặc 10.
- **Phân vị thứ 60 (6.0), Q3 (6.0)**: 60% và 75% dữ liệu có chất lượng ≤ 6, cho thấy phần lớn rượu có chất lượng trung bình hoặc thấp hơn.
- **IQR (1.0)**: Khoảng tứ phân vị nhỏ, cho thấy 50% dữ liệu ở giữa có sự phân tán thấp, chất lượng khá đồng đều.

## 4. Kiểm tra giá trị ngoại lai cho cột `quality`
Sử dụng IQR để xác định các giá trị ngoại lai.

In [8]:
# Tính Q1 và Q3
Q1 = np.quantile(df_dataset['quality'], 0.25)
Q3 = np.quantile(df_dataset['quality'], 0.75)
IQR = Q3 - Q1

# Giới hạn cho ngoại lai
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Tìm các giá trị ngoại lai
outliers = df_dataset[(df_dataset['quality'] < lower_bound) | (df_dataset['quality'] > upper_bound)]['quality']
print(f'Giới hạn ngoại lai: [{lower_bound}, {upper_bound}]')
print(f'Số lượng giá trị ngoại lai trong quality: {len(outliers)}')
print(f'Các giá trị ngoại lai: {outliers.unique()}')


Giới hạn ngoại lai: [3.5, 7.5]
Số lượng giá trị ngoại lai trong quality: 28
Các giá trị ngoại lai: [8 3]


**Nhận xét về giá trị ngoại lai**:
- Nếu có giá trị ngoại lai (ví dụ, quality = 3 hoặc 8), chúng có thể đại diện cho các loại rượu có chất lượng rất thấp hoặc rất cao, cần xem xét thêm về các thuộc tính đầu vào tương ứng.
- Nếu không có ngoại lai, điều này cho thấy dữ liệu chất lượng khá đồng nhất.